In [29]:
import pandas as pd
import os
import openai
from dotenv import load_dotenv, find_dotenv

import seaborn as sns

In [30]:
load_dotenv(find_dotenv())
openai.api_key = os.environ.get("openai_api")

In [45]:
df = pd.read_csv("../data/raw/SentimentLabeled_10112022.csv")
df = df[["text", "Bucket"]]
df["Relevant"] = df["Bucket"].apply(lambda x: x == "1")
df.drop(columns = ["Bucket"], inplace = True)
df.head()

,text,Relevant
0,I commend all of law enforcement for the secur...,False
1,"Socialist tyrant Maduro starves, tortures his ...",True
2,"Socialist tyrant Maduro starves, tortures his ...",False
3,Inhofe: The Canadians have already had convers...,False
4,Statement on Iran nuclear agreement. http://t....,False


In [41]:
df = pd.read_csv("../data/raw/SentimentLabeled_10112022.csv")
df[df["country"] == "Canada"].loc[15595]["text"]

'The President’s border-crossing permit for the A2A Railway Development Corp is a big boost for efforts to connect Alaska’s rich resources to a global market via freight rail through Canada. https://t.co/baSaeN9Lym'

In [32]:
df.head().iloc[0]["text"]

'I commend all of law enforcement for the security services provided for Pope Francis’ and Chinese President Xi’s visits to the U.S.'

In [46]:
def isRelevant(tweet):

    prompt = """
    You are a machine that will be given a Tweet from a U.S. politician, and you will be asked to determine its relevancy to one of three countries. If it is relevant, return "Relevant" and the specific country name. Otherwise, return "Irrelevant".
    
    The three possible countries are Canada, Iran, or China. 
    A "relevant" tweet would discuss how ONLY one of these countries' governments is having an impact on American politics.

    EXAMPLE TWEET:
    "They are also an assault on the American-led world order, and a disturbing premonition of an alternative world order—one controlled by the Chinese Communist Party and one that ends in Room 101."

    EXAMPLE ANSWER: 
    Relevant, China

    EXPLANATION:
    This tweet is clearly about the Chinese government and its impact over American politics. Hence, the return value is 1. 

    EXAMPLE TWEET:
    "JUST IN: House votes to block Obama from lifting Iran sanctions https://t.co/EFI5L9WjI4"

    EXAMPLE ANSWER: 
    Irrelevant

    EXPLANATION:
    This tweet, while mentioning Iran, does not reflect any substance over Iranian political effects on America. Its ONLY focus is on America, not Iranian impacts on America, and is thus irrelvant. 

    EXAMPLE TWEET:
    "The President’s border-crossing permit for the A2A Railway Development Corp is a big boost for efforts to connect Alaska’s rich resources to a global market via freight rail through Canada. https://t.co/baSaeN9Lym"

    EXAMPLE ANSWER: 
    Relevant, Canada

    EXPLANATION:
    This tweet is about how Canadian freight reils allow for American political expansion, and is thus relevant about one of our three options.
    
    TWEET: \n
    """

    prompt += tweet + "\n" + "ANSWER: \n\n"
    
    response = openai.Completion.create(engine = "text-davinci-003", prompt = prompt)
    answer = response["choices"][0]["text"].strip()
    
    return answer

def cleanAnswer(response):
   
    try:
        ans = response.split(",")
        return ans
    except:
        if "Irrelevant" in response:
            return response
        else:
            return "There was an error. The returned string was: [" + response + " ]"
            

In [47]:
subset = df.sample(n = 100)
subset.head()

subset["gpt_Relevant"] = subset["text"].apply(isRelevant)
subset["gpt_Relevant"] = subset["gpt_Relevant"].apply(cleanAnswer)


In [50]:
subset["gpt_Relevant"].value_counts()

[Relevant,  China]                                  66
[Relevant,  Iran]                                   17
[Irrelevant]                                         9
[Relevant,  Canada]                                  7
[Relevant,  Russia,  China,  North Korea,  Iran]     1
Name: gpt_Relevant, dtype: int64

In [34]:
for i in range(5):

    subset = df.sample(n = 100)
    subset.head()

    subset["gpt_Relevant"] = subset["text"].apply(isRelevant)
    subset["gpt_Relevant"] = subset["gpt_Relevant"].apply(cleanAnswer)

    accuracy = (subset["Relevant"] == subset["gpt_Relevant"]).mean()
    print("Trial #" + str(i + 1) + " accuracy: " + str(accuracy))

Trial #1 accuracy: 0.77
Trial #2 accuracy: 0.72
Trial #3 accuracy: 0.76
Trial #4 accuracy: 0.82
Trial #5 accuracy: 0.82


In [ ]:


# Tasks for Week 4 
# 1. Editing prompts (add more "relevance" explanation) 
# 2. Explore fine-tuning GPT-3 on Tweet data
# 3. BERT Tokenizer as control group (?)

